In [741]:
import pandas as pd
import sys
from top2vec import Top2Vec
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import punkt
from nltk.stem import WordNetLemmatizer
from collections import Counter
# import umap.umap_ as umap
# import umap.plot
nltk.download("stopwords")
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
import spacy
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim_models
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [742]:
def read_data():
    df = pd.read_csv("../data/training_data/complaint_classification_labeled_data_1_31_2023.csv")
    return df

In [743]:
df = read_data()

In [744]:
def split_rows_with_multiple_labels(df):
    df.loc[:, "label"] = (df.label.str.lower()
                                  .str.strip()
                                  .str.replace(r"#", "/", regex=False)
                                  .str.replace(r"internal misconduct\/administrative infraction",
                                               "internal misconduct; administrative infraction", regex=True)
    )
    df = (
        df.drop("label", axis=1)
        .join(
            df["label"]
            .str.split("/", expand=True)
            .stack()
            .reset_index(level=1, drop=True)
            .rename("label"),
            how="outer",
        )
        .reset_index(drop=True)
    )
    return df

In [745]:
df = df.pipe(split_rows_with_multiple_labels)

In [746]:
def extract_labels(df):
    dfa = df
    dfa.loc[:, "target"] = (df.label.str.lower()
                                    .str.strip()
                                    .str.replace(r"internal misconduct; administrative infraction", "", regex=False)
    )
    dfa = dfa[~((dfa.target.fillna("") == ""))]
    dfa.loc[:, "target"] = dfa.target.str.replace(r"(.+)", "0", regex=True)

    extract_targets = df.label.str.lower().str.strip().str.extract(r"(internal misconduct; administrative infraction)")

    df.loc[:, "target"] = extract_targets[0].str.replace(r"(.+)", "1", regex=True)
    df = df[~((df.target.fillna("") == ""))]

    df = pd.concat([df, dfa], axis=0)
    df = df.rename(columns={"text": "allegation_desc"})
    return df

In [747]:
df = df.pipe(extract_labels)

In [748]:
def split_data(df):
    training_data, test_data = train_test_split(df, test_size=0.3)
    return training_data, test_data, df

In [749]:
training_data, test_data, og_df = split_data(df)

training_data.to_csv("training_data/training_data.csv", index=False)
test_data.to_csv("test_data/test_data.csv", index=False)

In [750]:
training_data.target.value_counts()

0    915
1    165
Name: target, dtype: int64

In [751]:
test_data.target.value_counts()

0    405
1     58
Name: target, dtype: int64

In [752]:
################################################### top2vec ##################################################################################

In [753]:
def create_model(df):
    # unique = [x for x in df["allegation_topic_uid"]]
    df = [x for x in df["allegation_desc"]]

    model = Top2Vec(
        df,
        ngram_vocab=True,
        speed="deep-learn",
        use_embedding_model_tokenizer=True,
        min_count=5,
        # document_ids=unique
    )
    return model

In [754]:
def model():
    top2vec = create_model(og_df)
    return top2vec

In [755]:
model = model()

2023-02-01 14:17:18,097 - top2vec - INFO - Pre-processing documents for training
2023-02-01 14:17:18,343 - top2vec - INFO - Creating joint document/word embedding


KeyboardInterrupt: 

In [ ]:
# if len(model.get_topic_sizes()) > 1:
#     topic_words, word_scores, topic_nums = model.get_topics()
#     for words, scores, num in zip(topic_words, word_scores, topic_nums):
#         print(num)
#         print(f"Words: {words}")

In [ ]:
# topic_sizes, top_nums = model.get_topic_sizes()
# print(topic_sizes)
# print(top_nums)

In [ ]:
# documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=10)

# for doc, score, doc_id in list(zip(documents, document_scores, document_ids)):
#     print(f"Document: {doc_id}, Score: {score}")
#     print("--------------------")
#     print(doc)
#     print("--------------------")

In [ ]:
# documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=0, num_docs=20)

# ents = {(doc, score) for doc, score in list(zip(documents, document_scores))}
# df = pd.DataFrame(ents, columns=["allegation_desc", "score"])
# print(df)
# df.loc[:, "topic"] = "9"

In [ ]:
model.save("models/top2vec_train_model")

In [ ]:
# model.generate_topic_wordcloud(0)

In [ ]:
# model.topic_words

In [ ]:
# model.topic_words[0]

In [ ]:
# topic_words, word_scores, topic_nums = model.get_topics(0)
# for words, scores, nums in zip(topic_words, word_scores, topic_nums):
#   print("Topic Number: ",nums)
#   print(f"Words: {words}")
#   print("\n")

In [ ]:
# topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["inmate"], num_topics=0)
# for word, w_score, topic, t_score in list(zip(topic_words, word_scores, topic_scores, topic_nums)):
#     print(f"Word: \n{word}")
#     print("--------------------")
#     print(f"Word Score \n{w_score}")
#     print("--------------------")
#     print(f"Topic Score: \n{topic}")
#     print("--------------------")
#     print (f"Topic # \n{t_score}")
#     print("--------END---------")

In [ ]:
# model = Top2Vec.load("models/noso")

# umap_args = {
#     "n_neighbors": 15,
#     "n_components": 2, # 5 -> 2 for plotting 
#     "metric": "cosine",
# }
# umap_model = umap.UMAP(**umap_args).fit(model.topic_vectors)
# umap.plot.points(umap_model, labels=model.doc_top_reduced)

In [ ]:
############################################################ bertopic #################################################################

In [ ]:
docs = og_df.allegation_desc

In [ ]:
docs = json.loads(docs.to_json(orient='records'))

In [ ]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)
embeddings = np.array(embeddings)
embeddings = pd.DataFrame(embeddings).to_csv("vecs/bert_train_vecs.csv", index=False)

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", vectorizer_model=vectorizer_model, diversity=0.2)

In [ ]:
topic, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.save("models/bert_train_model")

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,364,-1_the_her_complainant_was
1,0,135,0_unprofessional_complainant_the_stated
2,1,111,1_failed_court_to_appear
3,2,64,2_camera_worn_body_activate
4,3,58,3_inmate_inmates_dorm_sick
5,4,42,4_bwc_activate_cew_uof
6,5,42,5_inmate_observed_food_giving
7,6,38,6_late_reported_duty_minutes
8,7,34,7_the_his_he_ex
9,8,34,8_did_not_write_report


In [ ]:
topic_model.get_topic(3)

[('inmate', 0.07901585088530677),
 ('inmates', 0.06516692049177385),
 ('dorm', 0.048291933678696504),
 ('sick', 0.046680020022922694),
 ('cell', 0.04654780035126704),
 ('observed', 0.04603910810061461),
 ('allowed', 0.044026122844746725),
 ('out', 0.04048334668066706),
 ('on', 0.03896511842464997),
 ('without', 0.038529606125760126)]

In [ ]:
topic_model.get_representative_docs(3)

['left his assigned area without permission at which time an inmate was able to open cells on dorm.',
 'deputy frazier allowed offender to come out of his cell for recreation time without notifying his supervisor.',
 'deputy frazier allowed offender to come out of his cell for recreation time without notifying his supervisor.']

In [ ]:
# topic_model.visualize_topics()

In [ ]:
# topic_model.visualize_barchart()

In [ ]:
df_bert = pd.DataFrame({"topic": topic, "documents": docs})

In [ ]:
df_bert

,topic,documents
0,0,the former chief of police received a complain...
1,-1,alleges officer burr made inappropriate facebo...
2,-1,refused an assignment.
3,27,left work without informing her rank.
4,21,failed to inform his supervisor that he would ...
...,...,...
1538,2,officer failed to activate his body-worn camer...
1539,13,officers accused of not making an arrest in a ...
1540,-1,complainant stated that the officer discrimina...
1541,-1,the complainant stated the officer was instruc...


In [ ]:
############################################################ gensim ##############################################################################

In [ ]:
train_docs = og_df.allegation_desc

In [ ]:
def lemmatization(descs, allowed_pos_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    final_text = []
    for desc in descs:
        doc = nlp(desc)
        new_text = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
        final_text.append(new_text)
    return (final_text)

In [ ]:
lemmatized_texts = lemmatization(docs)

In [ ]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [ ]:
data_words = gen_words(lemmatized_texts)

In [ ]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return list(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return list(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

# print(data_bigrams_trigrams[5])

In [ ]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

train_corpus = [id2word.doc2bow(text) for text in texts]
# print(train_corpus[0][0:20])

tdidf = TfidfModel(train_corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tdif = []

for i in range(0, len(train_corpus)):
    bow = train_corpus[i]
    low_value_words = []
    tdif_ids = [id for id, value in tdidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tdidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tdif
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tdif = [id for id in bow_ids if id not in tdif_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tdif]
    train_corpus[i] = new_bow

In [ ]:
# id2word = corpora.Dictionary(data_words)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

In [ ]:
gensim_model = gensim.models.ldamodel.LdaModel(corpus=train_corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                        #    per_word_topics=True,
                                           alpha="auto")
gensim_model.save('models/gensim_train.model')

In [ ]:
# gensim_model.print_topics(5, num_words=20)[:10]

In [ ]:
def gensim_vecs():
    gensim_train_vecs = []
    for i in range(len(train_docs)):
        top_topics = gensim_model.get_document_topics(train_corpus[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(20)]
        topic_vec.extend([len(train_docs.iloc[i])]) 
        gensim_train_vecs.append(topic_vec)
    return gensim_train_vecs

In [ ]:
gensim_vecs = gensim_vecs()
gensim_vecs = np.array(gensim_vecs)
gensim_vecs = pd.DataFrame(gensim_vecs).to_csv("vecs/gensim_train_vecs.csv", index=False)

In [ ]:
# test_doc = train_corpus[-1]

# vector = lda_model[test_doc]

# def Sort(sub_li):
#     sub_li.sort(key = lambda x: x[1])
#     sub_li.reverse()
#     return (sub_li)

# new_vector = Sort(vector)
# print(new_vector)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(gensim_model, train_corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.483596 -0.285522       1        1  50.904663
7     -0.439882  0.091966       2        1   7.721138
2      0.028766 -0.410485       3        1   5.096963
11    -0.167599 -0.337273       4        1   4.926976
10     0.089072  0.351107       5        1   3.348896
18    -0.125425  0.350897       6        1   3.347902
13    -0.277616  0.208132       7        1   3.338645
8     -0.223776 -0.055483       8        1   2.926921
17     0.203108 -0.293970       9        1   2.918521
12     0.364733 -0.103772      10        1   2.863635
6      0.185493 -0.137099      11        1   1.803161
14     0.297069  0.217510      12        1   1.798012
9     -0.057631 -0.070919      13        1   1.501952
1     -0.037484  0.057508      14        1   1.326146
16    -0.024950  0.166986      15        1   1.279587
4      0.112487  0.188636      16        1   1.165287
5      0.063591 -0.080401      17        1   1.122345
3      0.242080  0.026643      18        1   1.036324
19     0.168268  0.078840      19        1   0.941857
15     0.083291  0.036699      20        1   0.631071, topic_info=            Term         Freq        Total Category  logprob  loglift
32          fail   735.000000   735.000000  Default  30.0000  30.0000
7        officer  2887.000000  2887.000000  Default  29.0000  29.0000
315  complainant  2400.000000  2400.000000  Default  28.0000  28.0000
252       arrest   537.000000   537.000000  Default  27.0000  27.0000
45        accuse  1639.000000  1639.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
287          ask     0.043662     1.438548  Topic20  -7.8973   1.5706
855        drive     0.043658     1.438240  Topic20  -7.8974   1.5707
550        front     0.043660     1.438530  Topic20  -7.8974   1.5706
373      neglect     0.043666     1.439286  Topic20  -7.8972   1.5702
416         also     0.043661     1.438855  Topic20  -7.8974   1.5704

[769 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
179       8  0.472607  abandon
45        1  0.998997   accuse
238      14  0.816489    admit
146      11  0.370019   affair
186      13  0.731412    agent
...     ...       ...      ...
117      13  0.681498     wear
195      14  0.515453   weekly
27       10  0.981844     work
191      13  0.506601   wrrite
145      11  0.927403     year

[275 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 3, 12, 11, 19, 14, 9, 18, 13, 7, 15, 10, 2, 17, 5, 6, 4, 20, 16])